##Partie 2: Introduction à Federated Learning

Dans la dernière section, nous avons découvert les PointerTensors, qui créent l'infrastructure sous-jacente dont nous avons besoin pour préserver la confidentialité du Deep Learning. Dans cette section, nous allons voir comment utiliser ces outils de base pour implémenter notre premier algorithme Deep Learning préservant la confidentialité, Federated Learning.

Auteurs:
* Andrew Trask - Twitter: @iamtrask

###Qu'est-ce que Federated Learning?

C'est un moyen simple et puissant de former des modèles Deep Learning. Si vous pensez aux données de formation, c'est toujours le résultat d'une sorte de processus de collecte. Les gens (via des appareils) génèrent des données en enregistrant des événements dans le monde réel. Normalement, ces données sont agrégées dans un emplacement central unique afin que vous puissiez former un modèle d'apprentissage automatique. Federated Learning fait tourner la tête!

Au lieu d'apporter des données d'entraînement au modèle (un serveur central), vous apportez le modèle aux données d'entraînement (où qu'il se trouve).

L'idée est que cela permet à celui qui crée les données de posséder la seule copie permanente, et ainsi de garder le contrôle sur qui y a accès. Assez cool, hein?

##Section 2.1 - Un exemple Federated Learning de jouets

Commençons par former un modèle de jouet de manière centralisée. Il s'agit d'un simple que les modèles obtiennent. Nous devons d'abord:

* un jeu de données jouet
* un modèle
* une logique de formation de base pour la formation d'un modèle pour ajuster les données.

Remarque: Si cette API ne vous est pas familière, rendez-vous sur [fast.ai](https://www.fast.ai) et suivez leur cours avant de continuer dans ce didacticiel.

In [0]:
import torch
from torch import nn
from torch import optim

In [0]:
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# Un modèle de jouet
model = nn.Linear(2,1)

def train():
    # Logique de formation
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) effacer les dégradés précédents (s'ils existent)
        opt.zero_grad()

        # 2) faire une prédiction
        pred = model(data)

        # 3) calculer combien nous avons manqué
        loss = ((pred - target)**2).sum()

        # 4) déterminer quels poids nous ont fait manquer
        loss.backward()

        # 5) changer ces poids
        opt.step()

        # 6) imprimer nos progrès
        print(loss.data)

In [0]:
train()

Et voila! Nous avons formé un modèle de base de manière conventionnelle. Toutes nos données sont agrégées dans notre machine locale et nous pouvons les utiliser pour effectuer des mises à jour de notre modèle. Federated Learning, cependant, ne fonctionne pas de cette façon. Modifions donc cet exemple pour le faire de la manière de Federated Learning!

Alors, de quoi avons-nous besoin:

* créer un couple de travailleurs

* obtenir des conseils sur les données de formation sur chaque travailleur

* mise à jour de la logique de formation pour effectuer un apprentissage fédéré

Nouvelles étapes de formation:

  * envoyer le modèle au bon travailleur

  * former sur les données qui s'y trouvent

  * récupérer le modèle et répéter avec le prochain travailleur

In [0]:
import syft as sy
hook = sy.TorchHook(torch)

In [0]:
# créer un couple de travailleurs

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [0]:
# Un jeu de données jouet
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# obtenir des conseils sur les données de formation sur chaque travailleur en
# envoi de données d'entraînement à bob et alice
data_bob = data[0:2]
target_bob = target[0:2]

data_alice = data[2:]
target_alice = target[2:]

# Initaliser un modèle de jouet
model = nn.Linear(2,1)

data_bob = data_bob.send(bob)
data_alice = data_alice.send(alice)
target_bob = target_bob.send(bob)
target_alice = target_alice.send(alice)

# organiser les pointeurs dans une liste
datasets = [(data_bob,target_bob),(data_alice,target_alice)]

opt = optim.SGD(params=model.parameters(),lr=0.1)

In [0]:
def train():
    # Logique de formation
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(10):
        
        # NOUVEAU) parcourez l'ensemble de données de chaque travailleur
        for data,target in datasets:
            
            # NOUVEAU) envoyer le modèle au bon travailleur
            model.send(data.location)

            # 1) effacer les dégradés précédents (s'ils existent)
            opt.zero_grad()

            # 2) faire une prédiction
            pred = model(data)

            # 3) calculer combien nous avons manqué
            loss = ((pred - target)**2).sum()

            # 4) déterminer quels poids nous ont fait manquer
            loss.backward()

            # 5) changer ces poids
            opt.step()
            
            #NOUVEAU) obtenir le modèle (avec des dégradés)
            model.get()

            # 6) imprimer nos progrès
            print(loss.get()) # NOUVEAU) légère modification ... besoin d'appeler .get() en loss \
    
# federated averaging

In [0]:
train()

###Bien joué!
Et voilà! Nous formons maintenant un modèle de Deep Learning très simple en utilisant Federated Learning! Nous envoyons le modèle à chaque travailleur, générons un nouveau gradient, puis le rapportons à notre serveur local où nous mettons à jour notre modèle global. Jamais dans ce processus, nous ne voyons ou ne demandons l'accès aux données de formation sous-jacentes! Nous préservons l'intimité de Bob et Alice !!!


###Lacunes de cet exemple

Ainsi, bien que cet exemple soit une belle introduction à Federated Learning, il présente encore quelques lacunes majeures. Plus particulièrement, lorsque nous appelons model.get() et recevons le modèle mis à jour de Bob ou Alice, nous pouvons réellement en apprendre beaucoup sur les données d'entraînement de Bob et Alice en regardant leurs gradients. Dans certains cas, nous pouvons parfaitement restaurer leurs données d'entraînement!

Alors, que faire? Eh bien, la première stratégie que les gens utilisent est de faire la moyenne du gradient sur plusieurs individus avant de le télécharger sur le serveur central. Cette stratégie, cependant, nécessitera une utilisation plus sophistiquée des objets PointerTensor. Donc, dans la section suivante, nous allons prendre du temps pour en savoir plus sur les fonctionnalités de pointeur plus avancées, puis nous mettrons à niveau cet exemple Federated Learning.

##Toutes nos félicitations!!! - Il est temps de rejoindre la communauté!
Félicitations pour avoir terminé ce didacticiel pour ordinateur portable! Si cela vous a plu et que vous souhaitez rejoindre le mouvement vers la préservation de la vie privée, la propriété décentralisée de l'AI et la chaîne d'approvisionnement de l'AI (données), vous pouvez le faire de la manière suivante!

##Star PySyft sur GitHub
Le moyen le plus simple d'aider notre communauté est simplement de mettre en vedette les dépôts GitHub! Cela permet de faire connaître les outils sympas que nous construisons.

* [Star PySyft](https://github.com/OpenMined/PySyft)

##Joindrez notre Slack!
La meilleure façon de vous tenir au courant des dernières avancées est de rejoindre notre communauté! Vous pouvez le faire en remplissant le formulaire à http://slack.openmined.org

##Joindrez un projet de code!
La meilleure façon de contribuer à notre communauté est de devenir un contributeur de code! À tout moment, vous pouvez accéder à la page Problèmes de PySyft GitHub et filtrer pour "Projects". Cela vous montrera tous les billets de haut niveau donnant un aperçu des projets que vous pouvez rejoindre! Si vous ne souhaitez pas rejoindre un projet, mais que vous souhaitez faire un peu de codage, vous pouvez également rechercher d'autres mini-projets "uniques" en recherchant les problèmes GitHub marqués "good first issue".
* [PySyft Project](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
* [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

##Faire un don
Si vous n'avez pas le temps de contribuer à notre base de code, mais souhaitez tout de même apporter votre soutien, vous pouvez également devenir Backer sur notre Open Collective. Tous les dons vont à notre hébergement Web et à d'autres dépenses de la communauté telles que les hackathons et les rencontres!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)